## Notebook to extract the data for each student from VLE files

In [2]:
%reset -f

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

In [4]:
data_courses=pd.read_csv("courses.csv")
data_studentInfo=pd.read_csv("studentInfo.csv")
data_studentVle=pd.read_csv("studentVle.csv")
data_vle=pd.read_csv("vle.csv")
# data_studentAssessment=pd.read_csv("studentAssessment.csv")


In [5]:
data_studentInfo.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [10]:
%%time
student_clicks = data_studentVle.groupby(['id_student', 'code_module', 'code_presentation','date']).agg({'sum_click':['sum','mean']})
student_clicks.reset_index(level=[0,1,2,3], inplace=True)
student_clicks.columns = ['id_student', 'code_module', 'code_presentation','date', 'sum_clicks','mean_clicks']
student_clicks

Wall time: 3.59 s


,id_student,code_module,code_presentation,date,sum_clicks,mean_clicks
0,6516,AAA,2014J,-23,28,4.666667
1,6516,AAA,2014J,-22,82,11.714286
2,6516,AAA,2014J,-20,41,3.416667
3,6516,AAA,2014J,-17,7,1.750000
4,6516,AAA,2014J,-12,2,1.000000
...,...,...,...,...,...,...
1808114,2698588,BBB,2014J,240,3,1.000000
1808115,2698588,BBB,2014J,244,3,1.000000
1808116,2698588,BBB,2014J,248,5,2.500000
1808117,2698588,BBB,2014J,250,2,1.000000


### Calcaulting sum clicks and mean clicks at different percentage of course length for each student

In [7]:
vle_asmbl={"code_module":[],"code_presentation":[],"id_student":[]}
vle_asmbl.update({"sum_clicks0":[],"sum_clicks20":[],"sum_clicks40":[],"sum_clicks60":[],"sum_clicks80":[],"sum_clicks100":[]})
vle_asmbl.update({"mean_clicks0":[],"mean_clicks20":[],"mean_clicks40":[],"mean_clicks60":[],"mean_clicks80":[],"mean_clicks100":[]})

for i in tqdm(range(0,data_studentInfo.shape[0])):
        
    code_module=data_studentInfo.iloc[[i]]["code_module"].values[0]
    code_presentation=data_studentInfo.iloc[[i]]["code_presentation"].values[0]
    id_student=data_studentInfo.iloc[[i]]["id_student"].values[0]

    vle_asmbl["code_module"].append(code_module)
    vle_asmbl["code_presentation"].append(code_presentation)
    vle_asmbl["id_student"].append(id_student)

    dum=student_clicks[(student_clicks["code_module"]==code_module)&
                       (student_clicks["code_presentation"]==code_presentation)&
                       (student_clicks["id_student"]==id_student)]
    
    date_clicked=dum["date"].values
    cum_sum_clicks=dum["sum_clicks"].values
    cum_mean_clicks=dum["mean_clicks"].values
    
    ind=np.argsort(date_clicked)
    date_clicked=date_clicked[ind]
    cum_sum_clicks=np.cumsum(cum_sum_clicks[ind])
    cum_mean_clicks=np.cumsum(cum_mean_clicks[ind])/np.arange(1,date_clicked.shape[0]+1)
    
    module_presentation_length=data_courses[(data_courses["code_module"]==code_module)&
                                            (data_courses["code_presentation"]==code_presentation)]["module_presentation_length"].values[0]
    
    for prcnt in np.arange(0.0,1.01,0.20):

        ind=np.where(date_clicked/(module_presentation_length+1e-16)<=prcnt)

        if ind[0].shape[0]!=0:      
            vle_asmbl["sum_clicks"+str(int(prcnt*100))].append(cum_sum_clicks[ind[0][-1]])
            vle_asmbl["mean_clicks"+str(int(prcnt*100))].append(cum_mean_clicks[ind[0][-1]])
        else:
            vle_asmbl["sum_clicks"+str(int(prcnt*100))].append(0)
            vle_asmbl["mean_clicks"+str(int(prcnt*100))].append(0)

vle_asmbl=pd.DataFrame(vle_asmbl)
display(vle_asmbl)

100%|██████████████████████████████████████████████████████████████████████████| 32593/32593 [2:24:35<00:00,  3.76it/s]


,code_module,code_presentation,id_student,sum_clicks0,sum_clicks20,sum_clicks40,sum_clicks60,sum_clicks80,sum_clicks100,mean_clicks0,mean_clicks20,mean_clicks40,mean_clicks60,mean_clicks80,mean_clicks100
0,AAA,2013J,11391,147,528,628,713,791,934,6.904545,4.723002,4.206386,4.415033,4.424001,4.250956
1,AAA,2013J,28400,263,669,874,1063,1417,1435,3.287954,3.154050,2.857137,2.625161,2.798992,2.732375
2,AAA,2013J,30268,102,281,281,281,281,281,2.491162,3.469192,3.469192,3.469192,3.469192,3.469192
3,AAA,2013J,31604,169,799,1181,1550,1896,2158,3.624762,2.956942,3.139172,3.025253,2.986959,3.007250
4,AAA,2013J,32885,295,601,742,826,913,1034,3.954785,3.510630,3.285475,3.027455,2.813736,2.685387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,3,41,41,41,41,41,1.500000,2.027778,2.027778,2.027778,2.027778,2.027778
32589,GGG,2014J,2645731,0,62,84,516,868,893,0.000000,2.544118,2.186275,2.820069,2.854581,2.681378
32590,GGG,2014J,2648187,15,34,82,164,305,312,2.142857,1.546131,1.994615,2.100879,2.409217,2.326505
32591,GGG,2014J,2679821,65,149,275,275,275,275,3.380682,3.772727,3.675019,3.675019,3.675019,3.675019


### Saving the extracted vle data to vle_combined.csv file

In [8]:
vle_asmbl.to_csv('vle_combined.csv')